In [1]:
import numpy as np
import cv2 as cv
import time
from pathlib import Path
import depthai as dai


"""
    Captures grayscale images from a mono camera connected to the specified board socket.

    :param board_socket: Socket on the device where the camera is connected ("left" or "right").
    :param delay_ms: Delay in milliseconds between capturing consecutive images.
    :param num_images: Number of images to capture.
    """
def capture_images_from_mono_camera(board_socket="right", delay_ms=2000, num_images=10):

    if board_socket not in ["left", "right"]:
        print("Invalid board socket specified. Use 'left' or 'right'.")
        return

    # Define resolution for the mono camera
    sensor_resolution = dai.MonoCameraProperties.SensorResolution.THE_480_P

    # Create a pipeline
    pipeline = dai.Pipeline()

    # Define a mono camera node
    cam = pipeline.createMonoCamera()
    cam.setResolution(sensor_resolution)

    # Set the board socket for the camera
    if board_socket == "right":
        cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    else:
        cam.setBoardSocket(dai.CameraBoardSocket.LEFT)

    # Create XLink output for the camera
    xout = pipeline.createXLinkOut()
    xout.setStreamName(board_socket)
    cam.out.link(xout.input)

    # Connect to the device and start the pipeline
    with dai.Device(pipeline) as device:

        # Get the output queue for receiving frames
        q = device.getOutputQueue(name=board_socket, maxSize=4, blocking=False)

        # Create directory for saving images
        image_dir = Path(f"{board_socket}")
        image_dir.mkdir(parents=True, exist_ok=True)

        # Capture specified number of images
        for _ in range(num_images):
            # Get frame from the queue
            in_frame = q.get()
            frame = in_frame.getCvFrame()

            # Display and save the image
            cv.imshow(board_socket, frame)
            cv.imwrite(str(image_dir / f"{int(time.time() * 10000)}.jpg"), frame)
            cv.waitKey(delay_ms)

        cv.destroyAllWindows()
        device.close()




"""
    Captures color images from a color camera.

    :param delay_ms: Delay in milliseconds between capturing consecutive images.
    :param num_images: Number of images to capture.
    """
def capture_images_from_color_camera(delay_ms=2000, num_images=10):

    # Define resolution for the color camera
    sensor_resolution = dai.ColorCameraProperties.SensorResolution.THE_1080_P

    # Create a pipeline
    pipeline = dai.Pipeline()

    # Define a color camera node
    cam = pipeline.createColorCamera()
    cam.setResolution(sensor_resolution)

    # Create XLink output for the camera
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect to the device and start the pipeline
    with dai.Device(pipeline) as device:

        # Get the output queue for receiving frames
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Create directory for saving images
        image_dir = Path("rgb")
        image_dir.mkdir(parents=True, exist_ok=True)

        # Capture specified number of images
        for _ in range(num_images):
            # Get frame from the queue
            in_frame = q.get()
            frame = in_frame.getCvFrame()

            # Display and save the image
            imS = cv.resize(frame, (960, 540))  # Resize image
            cv.imshow("rgb", imS)
            cv.imwrite(str(image_dir / f"{int(time.time() * 10000)}.jpg"), frame)
            cv.waitKey(delay_ms)

        cv.destroyAllWindows()
        device.close()

In [2]:
import numpy as np
import cv2 as cv
from pathlib import Path


def calibrate_camera(images, source):
    if source not in {'right', 'left', 'rgb'}:
        print("Invalid source specified. Use 'right', 'left', or 'rgb'.")
        return

    # Termination criteria for corner detection
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points (chessboard corners)
    square_size = 0.03  # 3cm
    objp = np.zeros((6 * 9, 3), np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) * square_size

    # Arrays to store object points and image points
    objpoints = []  # 3D point in real-world space
    imgpoints = []  # 2D points in image plane

    not_found = []

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        ret, corners = cv.findChessboardCorners(gray, (9, 6), None)

        if ret:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

            cv.drawChessboardCorners(img, (9, 6), corners2, ret)
            cv.imwrite(f"{fname.split('.')[0]}_corners.jpg", img)
        else:
            not_found.append(fname)
            print(f"Corners not found for {fname}")

    for i in not_found:
        images.remove(i)

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    for fname in images:
        img = cv.imread(fname)
        h, w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)
        x, y, w, h = roi
        dst = dst[y:y + h, x:x + w]
        cv.imwrite(f"{fname.split('.')[0]}_result.jpg", dst)

    print("Saving camera matrix...")
    np.savetxt(f"{source}/camera_matrix.txt", mtx)

    print("Saving distortion vector...")
    np.savetxt(f"{source}/distortion_vector.txt", dist)

    print("Saving rotational vectors...")
    np.savetxt(f"{source}/rotational_vectors.txt", np.array(rvecs))

    print("Saving translation vectors...")
    np.savetxt(f"{source}/translation_vectors.txt", np.array(tvecs))

    print('Calibration completed successfully!')

In [3]:
# capturing images using right monochrome camera
capture_images_from_mono_camera('left')

/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_25044/2234083928.py:35: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.LEFT)
2024-04-20 20:02:15.323 Python[25044:1017576] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [4]:
import glob

right_images = glob.glob('left/*.jpg')

calibrate_camera(right_images, 'left')

Corners not found for left/17136577393243.jpg
Corners not found for left/17136577513793.jpg
Corners not found for left/17136577373100.jpg
Corners not found for left/17136577433414.jpg
Corners not found for left/17136577413302.jpg
Corners not found for left/17136577473621.jpg
Corners not found for left/17136577353022.jpg
Corners not found for left/17136577493672.jpg
Corners not found for left/17136577453551.jpg
Corners not found for left/17136577533896.jpg


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:3752: error: (-215:Assertion failed) nimages > 0 in function 'calibrateCameraRO'


In [6]:
def verify_calibration(calibration_images, camera_matrix, distortion_coeffs):
    for img_path in calibration_images:
        img = cv.imread(img_path)
        h, w = img.shape[:2]

        # Undistort the image
        undistorted_img = cv.undistort(img, camera_matrix, distortion_coeffs)

        # Display original and undistorted images side by side for comparison
        cv.imshow('Original Image', img)
        cv.imshow('Undistorted Image', undistorted_img)
        cv.waitKey(0)
        cv.destroyAllWindows()

# Load the camera matrix and distortion coefficients obtained from calibration
camera_matrix = np.loadtxt('right/camera_matrix.txt')
distortion_coeffs = np.loadtxt('right/distortion_vector.txt')

# Load calibration images
calibration_images = glob.glob('right/*.jpg')

# Verify calibration
verify_calibration(calibration_images, camera_matrix, distortion_coeffs)


FileNotFoundError: right/camera_matrix.txt not found.